In [41]:
# Importando pacotes necessários
import pandas as pd
import sqlalchemy
import os
!pip install pymysql
import pymysql
import sqlite3 as sql

In [8]:
# Criando a conexão com o MySQL
conn = pymysql.connect(
    host = 'localhost',
    port = int(3306),
    user = 'root',
    passwd = 'password12345',
    db = 'smarttbot',
)

In [92]:
# Importanto as Tabelas da Base de Dados
per_per_r = pd.read_sql_query("SELECT * FROM person_person_r",
    conn);
prod_prod_r = pd.read_sql_query("SELECT * FROM production_product_r",
    conn);
scr = pd.read_sql_query("SELECT * FROM sales_customer_r",
    conn);
ssodr = pd.read_sql_query("SELECT * FROM sales_sales_order_detail_r",
    conn);
ssohr = pd.read_sql_query("SELECT * FROM sales_sales_order_header_r",
    conn);
ssopr = pd.read_sql_query("SELECT * FROM sales_special_offer_product_r",
    conn);

In [15]:
# Checando se as Tabelas foram importadas
print(per_per_r);
print(prod_prod_r);
print(scr);
print(ssodr);
print(ssohr);
print(ssopr);

       BusinessEntityID PersonType Title FirstName MiddleName    LastName  \
0                     1         EM             Ken          J     Sánchez   
1                     2         EM           Terri        Lee       Duffy   
2                     3         EM         Roberto             Tamburello   
3                     4         EM             Rob                Walters   
4                     5         EM   Ms.      Gail          A    Erickson   
...                 ...        ...   ...       ...        ...         ...   
19967             20773         IN         Crystal                    Guo   
19968             20774         IN        Isabella          F  Richardson   
19969             20775         IN         Crystal          S          He   
19970             20776         IN         Crystal                  Zheng   
19971             20777         IN         Crystal                     Hu   

      Suffix  EmailPromotioN AdditionalContactInfo  \
0                    

In [ ]:
# Devido ao fato de eu ter mais familiaridade com o MySQL, tentei fazer a modelagem dos dados por lá.
# Tive diversos problemas para fazer importação e cheguei ao ponto de utilizar o Excel e o PowerBI para entender melhor os erros e os dados.
# Dito isto, utilizarei o Jupyter para fazer as queries que já haviam sido feitas no MySQL.

In [18]:
scr.describe

<bound method NDFrame.describe of        CustomerID  PersonID  StoreID  TerritoryID AccountNumber  \
0               1       NaN    934.0            1    AW00000001   
1               2       NaN   1028.0            1    AW00000002   
2               3       NaN    642.0            4    AW00000003   
3               4       NaN    932.0            4    AW00000004   
4               5       NaN   1026.0            4    AW00000005   
...           ...       ...      ...          ...           ...   
19815       30114    1985.0   1986.0            7    AW00030114   
19816       30115    1987.0   1988.0            6    AW00030115   
19817       30116    1989.0   1990.0            4    AW00030116   
19818       30117    1991.0   1992.0            4    AW00030117   
19819       30118    1993.0   1994.0            3    AW00030118   

                                    rowguid ModifiedDate  
0      3F5AE95E-B87D-4AED-95B4-C3797AFCB74F   2014-09-12  
1      E552F657-A9AF-4A7D-A645-C429D6E02491

In [49]:
!pip install -U pandasql

  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26781 sha256=83fc6beeaf9b240ac478f6614845fd617374d8bb1d78eab568515f37a08b55bc
  Stored in directory: c:\users\emili\appdata\local\pip\cache\wheels\63\e8\ec\75b1df467ecf57b6ececb32cb16f4e86697cbfe55cb0c51f07
Successfully built pandasql


In [51]:
from pandasql import sqldf
mysql = lambda q: sqldf(q, globals())

In [59]:
#Respondendo as Questões:
'''1. Escreva uma query que retorna a quantidade de linhas na tabela Sales.SalesOrderDetail
pelo campo SalesOrderID, desde que tenham pelo menos três linhas de detalhes.'''

mysql("SELECT SalesOrderID, COUNT(*) FROM ssodr WHERE SalesOrderID GROUP BY SalesOrderID HAVING SalesOrderID >= 3;")

,SalesOrderID,COUNT(*)
0,43659,12
1,43660,2
2,43661,15
3,43662,22
4,43663,1
...,...,...
31460,75119,3
31461,75120,3
31462,75121,3
31463,75122,2


In [71]:
'''2. Escreva uma query que ligue as tabelas Sales.SalesOrderDetail, Sales.SpecialOfferProduct 
e Production.Product e retorne os 3 produtos (Name) mais vendidos (pela soma de 
OrderQty), agrupados pelo número de dias para manufatura (DaysToManufacture).'''

mysql("SELECT ppr.ProductName AS Products, ssodr.OrderQty AS Qty FROM ssodr, prod_prod_r AS ppr INNER JOIN ssodr AS sod ON ppr.ProductID = sod.ProductID GROUP BY Products ORDER BY Qty LIMIT 3;")

PandaSQLException: (sqlite3.OperationalError) database or disk is full
[SQL: SELECT ppr.ProductName AS Products, ssodr.OrderQty AS Qty FROM ssodr, prod_prod_r AS ppr INNER JOIN ssodr AS sod ON ppr.ProductID = sod.ProductID GROUP BY Products ORDER BY Qty LIMIT 3;]
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [ ]:
#Infelizmente, não consegui rodar a query anterior (tanto no Jupyter quanto no MySQL) e não sei como melhora-la. Está errada mas fui até onde meu conhecimento me permite, atualmente.
#Sei que preciso melhorar e por isso sigo estudando mas peço desculpas por deixar essa incompleta.

In [81]:
"""3. Escreva uma query ligando as tabelas Person.Person, Sales.Customer e 
Sales.SalesOrderHeader de forma a obter uma lista de nomes de clientes e uma contagem 
de pedidos efetuados."""

mysql("SELECT ppr.FirstName || ' ' || ppr.MiddleName || ' ' || ppr.LastName AS FullName, COUNT(*) AS Qty FROM ssohr AS soh INNER JOIN scr AS sc ON soh.CustomerID = sc.CustomerID INNER JOIN per_per_r AS ppr ON PersonID = ppr.BusinessEntityID GROUP BY FullName ORDER BY Qty DESC;")

,FullName,Qty
0,Mason D Roberts,28
1,Dalton Perez,28
2,Samantha Jenkins,27
3,Ryan M Thompson,27
4,Nancy E Chapman,27
...,...,...
19096,Aaron Edwards,1
19097,Aaron Coleman,1
19098,Aaron Chen,1
19099,Aaron Butler,1


In [83]:
'''4. Escreva uma query usando as tabelas Sales.SalesOrderHeader, Sales.SalesOrderDetail e 
Production.Product, de forma a obter a soma total de produtos (OrderQty) por ProductID e 
OrderDate.'''

mysql("SELECT ppr.ProductID AS ProdID, ppr.ProductName, SUM(sod.OrderQty) AS Qty, soh.OrderDate FROM ssohr AS soh INNER JOIN ssodr AS sod ON soh.SalesOrderID = sod.SalesOrderID INNER JOIN prod_prod_r AS ppr ON sod.ProductID = ppr.ProductID GROUP BY ProdID ORDER BY Qty DESC;")

,ProdID,ProductName,Qty,OrderDate
0,712,AWC Logo Cap,8311,2011-05-31
1,870,Water Bottle - 30 oz.,6815,2013-05-30
2,711,Sport-100 Helmet Blue,6743,2011-05-31
3,715,Long-Sleeve Logo Jersey L,6592,2011-05-31
4,708,Sport-100 Helmet Black,6532,2011-05-31
...,...,...,...,...
261,898,LL Touring Frame - Blue 62,15,2013-05-30
262,911,LL Road Seat/Saddle,10,2013-05-30
263,943,LL Mountain Frame - Black 40,8,2013-05-30
264,942,ML Mountain Frame-W - Silver 38,7,2013-05-30


In [89]:
"""5. Escreva uma query mostrando os campos SalesOrderID, OrderDate e TotalDue da tabela 
Sales.SalesOrderHeader. Obtenha apenas as linhas onde a ordem tenha sido feita durante 
o mês de setembro/2011 e o total devido esteja acima de 1.000. Ordene pelo total devido 
decrescente."""

mysql("SELECT SalesOrderID, OrderDate, TotalDue FROM ssohr WHERE strftime('%m', OrderDate) = '07' AND TotalDue >= 1000 ORDER BY TotalDue DESC;")

,SalesOrderID,OrderDate,TotalDue
0,47395,2012-07-31,1650287482
1,47369,2012-07-31,1580565449
2,47355,2012-07-31,1457418553
3,43875,2011-07-01,1373432877
4,43884,2011-07-01,1304164829
...,...,...,...
2351,52751,2013-07-17,4409
2352,52816,2013-07-18,4409
2353,52992,2013-07-22,4409
2354,53160,2013-07-25,4409
